<a href="https://colab.research.google.com/github/motorio0829/Programming-for-AI-25-1/blob/main/Lecture10_Basic_analysis_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국복지패널데이터

* <https://www.koweps.re.kr/>
* 주관: 한국보건사회연구원
* 목적: 국내 가구의 경제활동을 연구하여 정책 지원에 반영할 목적으로 조사
* 데이터 전처리 필요 -> 데이터 전처리와 시각화를 연습해봅시다
* source: [데이터 시각화 with 파이썬](http://book.naver.com/bookdb/book_detail.naver?bid=18123838)


In [ ]:
# 코랩에서 한글 출력되도록
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 런타임 다시시작
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# data path
dpath = "/content/drive/MyDrive/data/"

## 데이터 읽기

* `welfare_python.csv`

variable | detail | code
:--- | :--- | :---
gender | 성별 | 1=남, 2=여
birth | 태어난 연도 | YYYY
marriage | 혼인상태 | 0=비해당(18세미만), 1=배우자있음, 2=사별, 3=이혼, 4=별거, 5=미혼(18세이상, 미혼모포함), 6=기타(사망 등)
religion | 종교 유무 | 1=있음, 2=없음
code_job | 직업 코드 | float64(xxx) (`welfare_job.csv` 참고)
income | 소득(월평균 임금) | float64(xxx)
code_region | 7개 권역별 지역구분 | 1=서울, 2=수도권(인천/경기), 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주



In [ ]:
# import data
# - gender : 성별(1=남, 2=여)
# - marriage : 혼인상태 (0=비해당(18세미만), 1=배우자있음, 2=사별, 3=이혼, 4=별거, 5=미혼(18세이상, 미혼모포함), 6=기타(사망 등))
df = pd.read_csv(dpath + "welfare_python.csv")
print(df.head())
print(df.info())

In [ ]:
# unicode error 뜨는 경우 encoding="CP949" 옵션 추가
ref = pd.read_csv(dpath + "welfare_job.csv", encoding="CP949")
ref.head()

## 데이터 전처리

1. gender : 1이면 '남성', 2이면 '여성'으로 변경하여 gender2로 코딩
2. marriage : 1이면 '결혼', 3이면 '이혼', 나머지는 '그외'로 변경하여 marriage2로 코딩
3. code_job : `welfare_job.csv'에서 한글로 된 직업이름정보(job)를 추출하여 병합하여 job으로 코딩
5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
6. religion : 1이면 '종교있음', 2이면 '무교'로 religion2로 표기
7. income : 소득이 없는 결측치의 경우 다른 월급들의 평균으로 대체 income2로 저장
8. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 후 범주형 변수로 변경하기

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (loc메소드 적용)
df.loc[df.gender==1, "gender2"] = "남성"
df.loc[df.gender==2, "gender2"] = "여성"
df.head()

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (dictionary and for문 이용)
dic = {1:"남성", 2:"여성"}
df["gender2"] = [dic[i] for i in df.gender]
df.head()

In [ ]:
# 2. marriage : 1이면 '결혼', 3이면 '이혼', 나머지는 '그외'라는 문자열로 처리하여 marriage2로 저장 (lambda함수 적용해보기)
def recoding(x):
    if x == 1:
        return "결혼"
    elif x == 3:
        return "이혼"
    else:
        return "그외"

df["marriage2"] = df.marriage.apply(lambda x :recoding(x))
# dataframe.column.apply(recording)
df.head()

In [ ]:
# 3. code_job : `welfare_job.csv'에서 한글로 된 직업이름정보(job)를 추출하여 병합 (merge이용)
df = pd.merge(df, ref, on="code_job")
df.head()

In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
df["age"] = 2024 - df.birth
df.head()

In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성 (현재 연도를 함수로 가져와보기)
# 구글에서 'year in python' 검색
import datetime
dt_now = datetime.datetime.now()
print(dt_now)
print(dt_now.year)


df["age"] = datetime.datetime.now().year - df.birth
df.head()

In [ ]:
# 6. religion : 1이면 '종교있음', 2이면 '무교'로 표기하여 religion2로 코딩 (한 줄 if else 구문 이용)
df["religion2"] = ["유" if i == 1 else "무" for i in df.religion]
df.religion2.value_counts()

In [ ]:
# 7. income : 소득이 없는 결측치의 경우 다른 월급들의 평균으로 대체하여 income2로 코딩
df.income.isna()      # 결측 확인 (isna() or isnull())
sum(df.income.isna()) # 결측치 개수 확인

df.income.mean()# 소득의 평균 계산
df["income2"] = df.income.fillna(df.income.mean())  # 결측치를 소득의 평균으로 대체

In [ ]:
df.head()

In [ ]:
# 8. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 (함수 이용)
def recoding(x):
    if x < 30:
        return "청년"
    elif x < 60:
        return "중년"
    else :
        return "노년"

df["agegroup"] = df.age.apply(recoding)
df.head()
df.agegroup.value_counts()

In [ ]:
df.head()

In [ ]:
# 8. 범주형 변수로 순서 카테고리 지정하기
df.agegroup = pd.Categorical(df.agegroup, categories=["청년", "중년", "노년"])
df.agegroup

In [ ]:
# 9. 다음 변수만 모아서 한글변수명을 붙이고 df2에 저장
# age=연령, agegroup=연령대, birth=생년, gender2=성별, marriage2=결혼상태, religion2=종교유무, job=직업, income2=월소득

df2 = df[["age", "agegroup", "birth", "gender2", "marriage2", "religion2", "job", "income2"]]
df2.head()

In [ ]:
df2.columns  # 컬럼명 확인

In [ ]:
# 위치가 변경될 수 있으니 dictionary 이용
dic = {"age":"연령", "agegroup":"연령대", "birth":"생년", "gender2":"성별", "marriage2":"결혼상태", "religion2":"종교유무",
       "job":"직업", "income2":"월소득"}
df2.rename(columns=dic, inplace=True)
df2.head()

In [ ]:
# df2를 /contents/drive/MyDrive/data/welfare_clean.csv로 저장
df2.to_csv(dpath + "welfare_clean.csv")

In [ ]:
# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기
df = pd.read_csv(dpath + "welfare_clean.csv")
df.head()

In [ ]:
# df2를 /contents/drive/MyDrive/data/welfare_clean.csv로 저장 (index 없이 저장하기)
df2.to_csv(dpath + "welfare_clean.csv", index=False)

# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기
df = pd.read_csv(dpath + "welfare_clean.csv")
df.head()

## 데이터 시각화

### 결혼상태와 종교유무의 분포

In [ ]:
# 결혼상태별 빈도 확인
df["결혼상태"].value_counts()

In [ ]:
# 빈도막대그래프 그리기
sns.catplot(kind="count", x="결혼상태", data=df);

In [ ]:
# 결혼상태 순서를 정해주기
df["결혼상태"] = pd.Categorical(df["결혼상태"], categories=["결혼", "이혼", "그외"])

In [ ]:
sns.catplot(kind="count", x="결혼상태", data=df);

In [ ]:
# countplot()의 order 옵션 사용해서 이혼, 결혼, 그외 순으로 그림 그려보기
sns.catplot(kind="count", x="결혼상태", data=df, order=["이혼", "결혼", "그외"]);

In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기
# 막대 색상을 팔레트 "Set2" 이용
sns.countplot(x="결혼상태", data=df, hue="종교유무", palette="Set2");

In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기 (종교 유 > 무 순으로 표기)
# 테두리 두께를 1로 지정하고, 테두리 색상을 navy로 주기
sns.countplot(x="결혼상태", data=df, hue="종교유무", hue_order=["유", "무"],
              linewidth=1, edgecolor="navy");

In [ ]:
# 위 그림을 가로막대로 그리고, 막대 색상을 팔레트 "Paired"로 주고, 테두리 색상을 gray로 주기
sns.countplot(y="결혼상태", data=df, hue="종교유무", palette="Paired", edgecolor="gray");

### 연령 분포


In [ ]:
# 연령에 대한 히스토그램
sns.displot(kind="hist", x="연령", data=df, binwidth=5);

In [ ]:
# 연령에 대한 히스토그램에 밀도곡선 얹고, 색상을 magenta로 지정하기
sns.displot(kind="hist", x="연령", data=df, binwidth=5, kde=True, color="m");

### 연령과 소득, 결혼유무 분포

In [ ]:
# 연령과 월소득의 산점도
sns.relplot(kind="scatter", x="연령", y="월소득", data=df);

In [ ]:
# 연령과 월소득의 산점도 그린 뒤, 마진에 히스토그램 얹기
sns.jointplot(x="연령", y="월소득", data=df);

In [ ]:
# pairplot으로 그림 그리기
sns.pairplot(data=df[["연령", "월소득"]]);

In [ ]:
# pairplot의 대각선은 밀도곡선으로 변경하고, 하삼각만 그리기
sns.pairplot(data=df[["연령", "월소득"]], diag_kind="kde", corner=True);

In [ ]:
# 연령과 월소득에 대한 pairplot을 결혼상태에 따라 색상을 구분하여 그리기
sns.pairplot(data=df[["연령", "월소득", "결혼상태"]], hue="결혼상태");

In [ ]:
# 연령과 월소득에 대한 산점도를 결혼상태에 따라 나눠서 그리되, 결혼상태는 결혼과 이혼만 추출해서 그리기
sns.relplot(kind="scatter", x="연령", y="월소득", data=df.query("결혼상태 == '결혼' | 결혼상태 == '이혼'"),
            col="결혼상태", col_order=["결혼", "이혼"]);

In [ ]:
# 연령에 따른 월소득을 회귀도표로 나타내되, 결혼상태에 따라 항목을 나누어서 그리고, 90% 신뢰구간 추가하기
sns.lmplot(x="연령", y="월소득", data=df, col="결혼상태", ci=90);

### 성별과 연령 분포

In [ ]:
# 연령에 대한 박스그림
sns.catplot(kind="box", x="연령", data=df, width=0.5);

In [ ]:
# 성별에 따라 연령에 대한 바이올린 그림 그리기
sns.violinplot(x="성별", y="연령", data=df);

In [ ]:
# 성별에 따른 연령분포에 대한 바이올린 그림을 결혼상태에 따라 나눠 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)
sns.violinplot(x="성별", y="연령", data=df.query("결혼상태 != '이혼'"), hue="결혼상태", hue_order=["결혼", "그외"]);

In [ ]:
# 성별에 따른 연령분포에 대한 바이올린 그림을 결혼상태에 따라 분할된 바이올린으로 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)
sns.violinplot(x="성별", y="연령", data=df.query("결혼상태 != '이혼'"), hue="결혼상태", hue_order=["결혼", "그외"], split=True);

### 성별과 소득 분포

In [ ]:
# 성별에 따른 평균 월소득 계산
df.groupby("성별")["월소득"].mean()

In [ ]:
df["월소득"].groupby(df["성별"]).mean()

In [ ]:
# 성별에 따른 평균 월소득에 대한 평균막대그래프 (신뢰구간 표시 안하기)
sns.catplot(kind="bar", x="성별", y="월소득", data=df, ci=None);

In [ ]:
# 성별에 따른 평균 월소득에 대한 평균막대그래프를 결혼상태에 따라 구분하여 가로막대로 그리기
sns.catplot(kind="bar", y="성별", x="월소득", data=df, ci=None, hue="결혼상태");

In [ ]:
# 성별에 따른 월소득에 대한 박스그림 그리기
sns.catplot(kind="box", x="성별", y="월소득", data=df);

In [ ]:
# 성별에 따른 월소득에 대한 박스그림 그리기 (평균 월소득이 400 이하인 개체만 추출)
sns.catplot(kind="box", x="성별", y="월소득", data=df.query("월소득<=400"));

In [ ]:
# 위 그림에 jitter형태로 데이터 표현하기 (색상은 검정, 점모양은 .으로 표현)
sns.catplot(kind="box", x="성별", y="월소득", data=df.query("월소득<=400"))
sns.stripplot(x="성별", y="월소득", data=df.query("월소득<=400"), color="k", marker=".");

# THE END